In [2]:
from utils.position_encodings import RotaryPositionEncoding, RotaryPositionEncoding3D, RotaryPositionEncoding2D
import torch

pos_enc_dim = 64 # has to be even
vision_feature_dim = 64

positional_encoder = RotaryPositionEncoding2D(feature_dim=pos_enc_dim)

xy = torch.rand(1, 2, 2)
xy_rel_enc = positional_encoder(xy)

vis_embeddings = torch.randn(1, 2, vision_feature_dim)

e1 = RotaryPositionEncoding.embed_rotary(vis_embeddings, xy_rel_enc[..., 0], xy_rel_enc[..., 1])
d1 = torch.dot(e1[0,0], e1[0,1])
print("torch.dot(e1[0,0], e1[0,1])", d1)


xy = xy + 10.
xy_rel_enc = positional_encoder(xy)
e2 = RotaryPositionEncoding.embed_rotary(vis_embeddings, xy_rel_enc[..., 0], xy_rel_enc[..., 1])
d2 = torch.dot(e2[0,0], e2[0,1])
print("torch.dot(e2[0,0], e2[0,1])", d2)
print("d1 == d1", torch.allclose(d1, d2))

torch.dot(e1[0,0], e1[0,1]) tensor(-5.1811)
torch.dot(e2[0,0], e2[0,1]) tensor(-5.1811)
d1 == d1 True


/home/felix/miniforge3/envs/tshape/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pos_enc_dim = 60 # has to be divisible by 3
vision_feature_dim = 60

positional_encoder = RotaryPositionEncoding3D(feature_dim=pos_enc_dim)

xy = torch.rand(1, 2, 3)
xy_rel_enc = positional_encoder(xy)
print("xy_rel_enc.shape", xy_rel_enc.shape)

vis_embeddings = torch.randn(1, 2, vision_feature_dim)

e1 = RotaryPositionEncoding.embed_rotary(vis_embeddings, xy_rel_enc[..., 0], xy_rel_enc[..., 1])
d1 = torch.dot(e1[0,0], e1[0,1])
print("torch.dot(e1[0,0], e1[0,1])", d1)


xy = xy + 10.
xy_rel_enc = positional_encoder(xy)
e2 = RotaryPositionEncoding.embed_rotary(vis_embeddings, xy_rel_enc[..., 0], xy_rel_enc[..., 1])
d2 = torch.dot(e2[0,0], e2[0,1])
print("torch.dot(e2[0,0], e2[0,1])", d2)
print("d1 == d1", torch.allclose(d1, d2))

xy_rel_enc.shape torch.Size([1, 2, 60, 2])
torch.dot(e1[0,0], e1[0,1]) tensor(5.2609)
torch.dot(e2[0,0], e2[0,1]) tensor(5.2609)
d1 == d1 True
